In [1]:
import getpass
import requests
import sys
import http.client as http_client
import logging
import json
import pandas as pd
import numpy as np
from jaqpotpy import Jaqpot

In [ ]:
user = "Hackathon"

pw = getpass.getpass("Login password for user '{}': ".format(user))

url = "https://ssl.biomax.de/nanocommons/bioxm/rest/api"

proxies = {
  #'https': 'server:8080'
}

print("Opening session...")
response = requests.get(url + "/createUserSessionSimple?name={}&password={}".format(user, pw),
                        proxies=proxies)

session_id = ""
if response.status_code == 200:
    session_id = response.text
    print("Opened session " + session_id)
else:
    print("Failed to open session: " + str(response.text))
    sys.exit(1)

request_data= """
  {
    "SearchResultWithReportsRequest":
    {
      "firstIndex": 0,
      "maxCount": 100,
      "local": True,
      "viewName": "General - RestAPI - List of Particles With a Variable With a Value",
      "query": {
          "queryName": "General - RestAPI - List of Particles With a Variable With a Value",
          "variableValues": [{"id": "Variable", "value": "*"}, {"id": "Value", "value": "10"}]
      }
    }
  }
"""

try:               
    headers = {'Content-type': 'application/json'}

    http_client.HTTPConnection.debuglevel = 1

    logging.basicConfig()
    logging.getLogger().setLevel(logging.DEBUG)
    requests_log = logging.getLogger("requests.packages.urllib3")
    requests_log.setLevel(logging.DEBUG)
    requests_log.propagate = True

    response = requests.post(url + "/getSearchResultWithReports?userSessionId=" + session_id, data=request_data,
                           headers=headers, proxies=proxies)
    print("Status: " + str(response.status_code))

    if (response.status_code == 200):
        print(response.text)
        json_data2 = json.loads(response.text)
        json_dataset2 = json_data['SearchResultWithReports']['objectReports']

    else:
        print("ERROR:")
        print(response.text)


finally:
    response = requests.get(url + "/destroyUserSession?userSessionId=" + session_id, proxies=proxies)
    if (response.status_code == 200):
        print("\nClosed session.")
    else:
        print("\nError: Failed to close session " + session_id)
        
        
data_dict = {}
data_dict['Id'] = []
for dato in json_data2['SearchResultWithReports']['objectReports']:
    for atr in dato['attributes']:
        data_dict[atr['name']] = []

  
for dato in json_data2['SearchResultWithReports']['objectReports']:
    data_dict['Id'].append(dato["objectURL"])
    for key, value in data_dict.items():
        for atr in dato['attributes']:
            if atr['name'] == key:
                data_dict[key].append(atr['values'])

            
    
df = pd.DataFrame(data_dict)
print(df)

jaqpot = Jaqpot("https://api.jaqpot.org/jaqpot/services/")

jaqpot.request_key_safe()

jaqpot.upload_dataset(df=df, id='Id', title="Fisrt from biomax", description="First attempt to upload dataset from biomax")
